# merge_hourly_standardization debugging 
Demonstrate issues with merge_hourly_standardization function. <br>Date: May 18, 2025

In [1]:
import xarray as xr 
import pandas as pd
import logging

from merge_hourly_standardization import merge_hourly_standardization
from merge_derive_missing_vars import merge_derive_missing_vars

### 1. Read in a sample zarr, setup for input to function
1. Make a simple logger that just prints to the console (required input to function)
2. Read in zarr as xr.Dataset 
3. Convert xr.Dataset --> pd.DataFrame (required input to function)
4. Change MultiIndex DataFrame to a flat (single-level) DataFrame, with `time` and `station` as normal columns 
5. Input DataFrame, variable attributes, and logger to `merge_hourly_standardization`

In [ ]:
# Create a simple logger that just prints to the console
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()  

# Read in zarr as xarray object 
s3_path = "s3://wecc-historical-wx/3_qaqc_wx/ASOSAWOS/ASOSAWOS_69007093217.zarr/"
ds = xr.open_zarr(s3_path)
var_attrs = {
        var: ds[var].attrs for var in list(ds.data_vars.keys())
    }  # Attributes from each variable

# Convert to dataframe 
df = ds.to_dataframe()

# Set index MultiIndex --> normal df 
df.reset_index(inplace=True)

# Input dataframe to merge_hourly_standardization function 
df_after_merge, var_attrs = merge_hourly_standardization(df, var_attrs, logger)

# Derive missing vars 
df_after_missing_vars = merge_derive_missing_vars(df_after_merge)

INFO:aiobotocore.credentials:Found credentials in environment variables.
/Users/nicolekeeney/opt/anaconda3/envs/hist-obs/lib/python3.10/site-packages/pandas/core/arrays/timedeltas.py:908: RuntimeWarning: invalid value encountered in cast
  base = data.astype(np.int64)
/Users/nicolekeeney/opt/anaconda3/envs/hist-obs/lib/python3.10/site-packages/pandas/core/arrays/timedeltas.py:912: RuntimeWarning: invalid value encountered in cast
  data = (base * m + (frac * m).astype(np.int64)).view("timedelta64[ns]")


### 2. Observe the original DataFrame
Note how `station` and `time` are columns 

In [ ]:
df

### 3. Obeserve the output DataFrames

In [ ]:
# df after merge_hourly_standardization function 
df_after_merge

In [ ]:
# df after merge_derive_missing_vars function 
df_after_merge